In [ ]:
id = 'loading-bay'
spektrix_event_ids = ''
venue_postcode = ''
rosterfy_event_ids = ''
project_ids = ''
venue_ids = 'rec4maUwKspPcDxNl recFYiOZvNnYDgbfk recInFDrcQ4KutUcL recIrQ1v64zKajXU1 recalzwrAJkBYOz8w'

id = 'beacon'
spektrix_event_ids = ''
venue_postcode = ''
rosterfy_event_ids = ''
project_ids = ''
venue_ids = 'rec7tn1K6r85N72yc recW1DV5uXWsrUShy recea0GEE29eKCtxu recx817GdO7wmJDf6'

In [ ]:
id: str
spektrix_event_ids: str
venue_postcode: str
rosterfy_event_ids: str
project_ids: str
venue_ids: str

In [ ]:
import json
from datetime import datetime

from utils.paths import SITE
from utils.process.event import Programme, Sustainability, Tickets, Volunteers

In [ ]:
OUTPUT_DIR = SITE / 'events' / id / '_data/event'
PROGRAMME_DIR = OUTPUT_DIR / 'programme'
PROGRAMME_DIR.mkdir(exist_ok=True, parents=True)
TICKETS_DIR = OUTPUT_DIR / 'tickets'
TICKETS_DIR.mkdir(exist_ok=True, parents=True)
VOLUNTEERS_DIR = OUTPUT_DIR / 'volunteers'
SUSTAINABILITY_DIR = OUTPUT_DIR / 'sustainability'

In [ ]:
if project_ids or venue_ids:
    programme = Programme(project_ids=project_ids.split(), venue_ids=venue_ids.split())
    with open(PROGRAMME_DIR / 'summary.json', 'w') as f:
        json.dump(programme.summarise(), f, indent=2)
    (
        programme.event_reports
        .aggregate(['Venue Name', 'Project Name'], {
            'start_date': ('event_date', min),
            'audience': ('audience', sum),
            'participants': ('participants', sum),
        })
        .rename({ 'Venue Name': 'venue_name', 'Project Name': 'project_name' })
        .sort('start_date')
        .convert(['start_date'], lambda x: x.isoformat())
        .tojson(PROGRAMME_DIR / 'total_by_venue.json', indent=2)
    )

In [ ]:
if spektrix_event_ids:
    tickets = Tickets(spektrix_event_ids.split())
    tickets.detailed().tocsv(TICKETS_DIR / 'by_geography.csv')
    tickets.summarise().tocsv(TICKETS_DIR / 'summary.csv')
    tickets.types().tocsv(TICKETS_DIR / 'by_type.csv')

In [ ]:
if rosterfy_event_ids:
    VOLUNTEERS_DIR.mkdir(exist_ok=True, parents=True)
    volunteers = Volunteers(str(rosterfy_event_ids).split())
    volunteers.summarise().tocsv(VOLUNTEERS_DIR / 'summary.csv')

In [ ]:
if project_ids:
    SUSTAINABILITY_DIR.mkdir(exist_ok=True, parents=True)
    sustainability = Sustainability(project_ids.split()).summarise()
    sustainability.tocsv(SUSTAINABILITY_DIR / 'breakdown.csv')
    sustainability.select(lambda r: r.impact_type == 'ALL' and r.scope == 'ALL').cut('tCO2e').tojson(SUSTAINABILITY_DIR / 'summary.json', lines=True)